# Intro to Google Earth Engine

**To run the code:**
> Make sure you have this notebook open in Google Colab (if you are starting from the digital textbook, click on the {fa}`rocket` icon and click **Colab**). Each block of code is called a cell. To run a cell, hover over it and click the arrow in the top left of the cell, or click inside of the cell and press Shift + Enter.

Note: When you run a block of code for the first time, Google Colab will say `Warning: This notebook was not authored by Google`. Please click `Run Anyway`.

A great feature of Google Colab is that you are able to write Python code and see the output directly on your browser. Let's go through the basics below:

In [1]:
import ee

# Authenticate Google Earth Engine
ee.Authenticate()

# Change "emerge-lessons" to your project ID if it is different
ee.Initialize(project="emerge-lessons")

In the code above, change "emerge-lessons" to your own project ID. For example, if your ID is "emerge-34956," you can change it to the following:
```
ee.Initialize(project="emerge-34956")
```

### Basic Terminology - Using Collections in GEE

**Features** are geometric objects with a list of properties
> ee.Feature

**Images** are similar to features, but can have several bands

> ee.Image

**Collections** are groups of features or images

> ee.FeatureCollection or ee.ImageCollection

When you go into the details of an individual dataset, you can find the snippet of how to import that specific dataset

For this example, we'll be using the [MODIS Terra Land Surface Temperature (LST)](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1) dataset.

In [2]:
# Import the MODIS land surface temperature collection
lst = ee.ImageCollection('MODIS/061/MOD11A1')

Datasets include several sets of information across several bands, with some datasets containing daily imagery in a 1km resolution (like the MODIS LST dataset we’re using) to one image a year in a 30m resolution for others.

The LST Collection uses these bands


*   LST_Day_1km: Daytime Land Surface Temperature
*   Day_view_time: Local time of day observation
* LST_Night_1km: Nighttime Land Surface Temperature

With so many images within the collection, we need to filter those images using dates

Use either of these for filtering
```
filterDate()
select()
```

In [3]:
# Initial date of interest (inclusive)
i_date = '2024-01-01'

# Final date of interest (exclusive)
f_date = '2024-01-31'

# Selection of appropriate bands and dates for LST
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

Next, we’ll define two points of interest (POIs) in relation to what we want to look at. Since we’re looking at Land Surface Emissivity (land temp), we’ll make a comparison of the temperature in an urban area versus a rural area.

In [4]:
# Define the urban location of interest as a point near Miami, Florida
urban_lon = -80.196432
urban_lat = 25.779766
urban_poi = ee.Geometry.Point(urban_lon, urban_lat)

# Define the rural location of interest as a point away from the city in Homestead, Florida
rural_lon = -80.4998113
rural_lat = 25.3933527
rural_poi = ee.Geometry.Point(rural_lon, rural_lat)

The MODIS emissivity dataset that we're using should be corrected by a scale of 0.02 to get units of Kelvin. You can find this in the [Google Earth Engine catalogue at this link](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD11A1#bands).

*Remember to convert your values for a final answer!*

We need the average land temperature of the two POIs we're looking at, so we'll use `mean()`

In [5]:
# Scale in meters
scale = 1000

# Function to convert LST to Celsius and get mean LST
def get_mean_lst(poi, scale):
    mean_lst = lst.mean().sample(poi, scale).first().get('LST_Day_1km').getInfo()
    return round(mean_lst * 0.02 - 273.15, 2)  # Convert to Celsius

# Get mean LST for both urban and rural points
urban_lst = get_mean_lst(urban_poi, scale)
rural_lst = get_mean_lst(rural_poi, scale)

print ("The mean land surface temperature for the urban point is", urban_lst, "°C")
print ("The rural  land surface temperature for the urban point is", rural_lst, "°C")

The mean land surface temperature for the urban point is 23.15 °C
The rural  land surface temperature for the urban point is 20.9 °C


## Interactive Maps

In [6]:
import folium
import geemap

Create an empty map zoomed to Florida. You can click and drag to move around the map, as well as zoom in and out.

In [7]:
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)
display(map)

Next, we define a function (from [this tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)) to add Google Earth Engine data to a map in a way that allows it to be interactively displayed.

In [8]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [9]:
# Visualization parameters used to set the colors for the data
lst_vis = {
    'min': 13000.0,
    'max': 16500.0,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}

# Add the data to the map
map.add_ee_layer(lst.mean().select('LST_Day_1km'), lst_vis, "LST")

display(map)

References
- [Creating Interactive Maps with geemap](https://book.geemap.org/chapters/02_maps.html)
- [feedbackAn Intro to the Earth Engine Python API](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)